In [1]:
import os
import shutil
import cv2
import math
import matplotlib.pyplot as plt

import numpy as np
from scipy import stats
from PIL import Image

import torch
import torchvision
from  torchvision import transforms
from google.colab.patches import cv2_imshow
import itertools

# Загрузка данных и кадрирование

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
video_model_path = '/content/drive/MyDrive/SF/Project_5/data/coach.mp4'
video_person_path = '/content/drive/MyDrive/SF/Project_5/data/student.mp4'

In [4]:
def get_frames(video_file, folder_name):
  if not os.path.exists(folder_name):
    os.makedirs(folder_name)
  count = 0
  cap = cv2.VideoCapture(video_file)   # загрузка видео 
  frameRate = cap.get(5) # частота кадров
  while(cap.isOpened()):
    frameId = cap.get(1) # номер текущего кадра
    ret, frame = cap.read()
    if (ret != True):
        break
    elif (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count
        count+=1
        directory = os.path.join(folder_name, filename)
        cv2.imwrite(directory, frame)

  cap.release()

In [5]:
get_frames(video_model_path, 'model')
get_frames(video_person_path, 'person')

In [6]:
num_foto = sum(os.path.isfile(os.path.join('person', f)) for f in os.listdir('person'))
sigmas = np.array([
    .26, .25, .25, .35, .35, .79, .79, .72, .72, .62, .62, 1.07, 1.07, .87,
    .87, .89, .89]) / 10.0

# Загрузка модели и получение предсказания


In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
weights='KeypointRCNN_ResNet50_FPN_Weights.DEFAULT'
model_keypointrcnn = torchvision.models.detection.keypointrcnn_resnet50_fpn(progress=True, weights=weights)
model_keypointrcnn.to(device)

Downloading: "https://download.pytorch.org/models/keypointrcnn_resnet50_fpn_coco-fc266e95.pth" to /root/.cache/torch/hub/checkpoints/keypointrcnn_resnet50_fpn_coco-fc266e95.pth


  0%|          | 0.00/226M [00:00<?, ?B/s]

KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.

In [8]:
prediction_model = []
prediction_person = []

transform = transforms.Compose([transforms.ToTensor()]) 

for foto in range(num_foto):

    path_model = '/content/model/frame'+ str(foto)+'.jpg'
    path_person = '/content/person/frame'+ str(foto)+'.jpg'

    image_model = Image.open(path_model).convert("RGB")
    img_tensor_model = transform(image_model)
    img_tensor_model = img_tensor_model.unsqueeze(0)

    image_person = Image.open(path_person).convert("RGB")
    img_tensor_person = transform(image_person)
    img_tensor_person = img_tensor_person.unsqueeze(0)

    model_keypointrcnn.eval()

    img_tensor_model.to(device)
    img_tensor_person.to(device)
    with torch.no_grad():
      prediction_model.append(model_keypointrcnn(img_tensor_model)[0])
      prediction_person.append(model_keypointrcnn(img_tensor_person)[0])

#Оценка соответствия поз 

In [9]:
def get_keypoints(output_model, keypoint_threshold=2, conf_threshold=0.9):
    all_keypoints = output_model['keypoints']
    all_scores = output_model['keypoints_scores']
    confs = output_model['scores']
    best_keypoints_img = []
    bed_keypoints_img = []
    # для каждого задетектированного человека
    for person_id in range(len(all_keypoints)):
        # проверяем степень уверенности детектора
        if (confs[person_id] > conf_threshold):
            # собираем ключевые точки конкретного человека
            keypoints = all_keypoints[person_id, ...]
            # собираем скоры для ключевых точек
            scores = all_scores[person_id, ...]
            # итерируем по каждому скору
            keypoints_best_pose = []
            bed_keypoints_pose = []
            for kp in range(len(scores)):
                # запоминаем индексы опорных точек с низкой степенью уверенности
                if scores[kp] <= keypoint_threshold:
                  bed_keypoints_pose.append(kp)
                # конвертируем массив ключевых точек в список целых чисел
                keypoint = list(
                    map(int, keypoints[kp, :2].detach().numpy())
                )
                keypoints_best_pose.append(keypoint)
            # список ключевых точек с высокой/низкой уверенностью детекции    
            best_keypoints_img.append(np.array(keypoints_best_pose))
            bed_keypoints_img.append(np.array(bed_keypoints_pose))
            
    # форматирование списков для работы
    bed_keypoints_img = np.array(bed_keypoints_img)       
    if bed_keypoints_img.shape[0] == 1:
      bed_keypoints_img = bed_keypoints_img.squeeze(0)
    
    best_keypoints_img = np.array(best_keypoints_img)       
    if best_keypoints_img.shape[0] == 1:
      best_keypoints_img = best_keypoints_img.squeeze(0)
      
    return best_keypoints_img, bed_keypoints_img

In [133]:
# Соберём два набора ключевых точек
model_keypoints = []
person_keypoints = []
# Индексы сохраненных и неучтённых фото
id_pose = []
id_cut = []
sigmas_cut = []
for foto in range(num_foto):
    model_poses, idx_pass_model = get_keypoints(prediction_model[foto])
    person_poses, idx_pass_person = get_keypoints(prediction_person[foto])
    
    # отбор фото, где количество достоверно опознанных тренеров равно
    # количеству достоверно опознанных учеников
    try:
      #model_poses[0].ndim == person_poses[0].ndim:
      # индексы опорных точек, детерминируемых с низким score у обоих поз
      idx_pass = np.concatenate((idx_pass_model,idx_pass_person)).astype(int)

      # удаление в обоих наборах опорных точек,
      # отсутвующих либо у тренера, либо у ученика
      model_poses_cut = np.delete(model_poses, np.unique(idx_pass), axis = 0)
      person_poses_cut = np.delete(person_poses, np.unique(idx_pass), axis = 0)
    
      # удаление констант, связанных с удаляемыми опорными точками
      sigmas_cut.append(np.delete(sigmas, np.unique(idx_pass),axis = 0))

      # номера фото, участвующих в оценке
      id_pose.append(foto)

      # два набора ключевых точек для оценки
      model_keypoints.append(model_poses_cut)
      person_keypoints.append(person_poses_cut)
    except:
      # номера фото, не участвующих в оценке
      # на одной фото найдено больше одной позы, не найдено ни одной позы
      id_cut.append(foto)

<ipython-input-9-78a038eb5899>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bed_keypoints_img = np.array(bed_keypoints_img)


In [111]:
def affine_transformation(person_pose, model_pose):
  # С помощью расширенной матрицы можно осуществить умножение вектора x
  # на матрицу A и добавление вектора b за счёт единственного матричного умножения.
  # Расширенная матрица создаётся путём дополнения векторов "1" в конце.
  pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
  unpad = lambda x: x[:, :-1]
  
  # Расширим наборы ключевых точек до [[ x y 1] , [x y 1]]
  Y = pad(model_pose)
  X = pad(person_pose)

  # Решим задачу наименьших квадратов X * A = Y
  # и найдём матрицу аффинного преобразования A.
  A, res, rank, s = np.linalg.lstsq(X, Y)
  A[np.abs(A) < 1e-10] = 0  # превратим в "0" слишком маленькие значения

  # Преобразование входного набора ключевых точек с помощью матрицы А
  A_transform = lambda x: unpad(np.dot(pad(x), A))
  input_transform = A_transform(person_pose)

  return input_transform

In [112]:
def cos_similarity(pose1, pose2):
    # косинусное сходство между всеми ключевыми точками размерностью = 17*17
    cossim = pose1.dot(np.transpose(pose2)) / (
        np.linalg.norm(pose1, axis=1) * np.linalg.norm(pose2, axis=1)
    )
    # косинусное сходство между советующими ключевыми точками
    cossim_pair = np.diagonal(cossim)

    # усредненное косинусное сходство по фигуре
    cossim_person = cossim_pair.mean()

    return cossim_person

In [113]:
def compute_oks(sigmas, model_bb, input_keypoints, model_keypoints):
    # OKS = exp(-d^2/(2*s^2*k^2))/number_keypoints
    
    k = 2 * sigmas

    # площадь bounding box
    s = abs(model_bb[2] - model_bb[0]) * abs(model_bb[3] - model_bb[1])

    # расстояние по каждой координате (катеты)
    distance = np.subtract(model_keypoints, input_keypoints)

    # евклидово расстояние в квадрате
    d_square = np.sum(distance**2, axis=1)
    
    degree = d_square/(2* s**2 * k**2)

    e = np.exp(-degree)

    return np.mean(e) 

In [134]:
bounding_box = []
for foto in id_pose:  
  # выбор bounding box у значимой позы
  idx_bool = prediction_model[foto]['scores']>0.9
  idx_int = idx_bool.nonzero()
  idx = int(idx_int)
  bb = torch.Tensor.numpy(prediction_model[foto]['boxes'][idx])
  bounding_box.append(bb) 

In [135]:
person_keypoint_aff = []
for foto in range(len(id_pose)):
  keypoint_aff = affine_transformation(person_keypoints[foto],
                                       model_keypoints[foto])
  person_keypoint_aff.append(keypoint_aff)

<ipython-input-111-ba605e515d4d>:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  A, res, rank, s = np.linalg.lstsq(X, Y)


In [136]:
cos_sim = []
oks = []
cos_sim_print = []
oks_print = []
for foto in range(len(id_pose)):
  cos_sim_pose = cos_similarity(person_keypoint_aff[foto], model_keypoints[foto])
  oks_pose = compute_oks(sigmas_cut[foto], bounding_box[foto], person_keypoint_aff[foto], model_keypoints[foto])

  cos_sim_pose_print = str(f"{cos_sim_pose:.0%}")
  oks_pose_print = str(f"{oks_pose:.0%}")

  cos_sim.append(cos_sim_pose)
  oks.append(oks_pose)

  cos_sim_print.append(cos_sim_pose_print)
  oks_print.append(oks_pose_print)


In [137]:
message = "error"
for id in id_cut:
  cos_sim_print.insert(id, message)
  oks_print.insert(id, message)

# Визуализация keypoint и результатов оценки

In [138]:
def draw_keypoints_per_person(img, all_keypoints, all_scores, confs, thickness, keypoint_threshold=2, conf_threshold=0.9):
    # создаём спектр цветов
    cmap = list(itertools.permutations((200, 100, 0), 3))
    # создаём копию изображений
    img_copy = img.copy()
    # для каждого задетектированного человека
    for person_id in range(len(all_keypoints)):
        # проверяем степень уверенности детектора
        if confs[person_id] > conf_threshold:
            # собираем ключевые точки конкретного человека
            keypoints = all_keypoints[person_id, ...]
            # собираем скоры для ключевых точек
            scores = all_scores[person_id, ...]
            # итерируем по каждому скору
            for kp in range(len(scores)):
                # проверяем степень уверенности детектора опорной точки
                if scores[kp] > keypoint_threshold:
                    # конвертируем массив ключевых точек в список целых чисел
                    keypoint = tuple(
                        map(int, keypoints[kp, :2].detach().numpy().tolist())
                    )
                    # рисуем круг радиуса thickness вокруг точки
                    cv2.circle(img_copy, keypoint, thickness, cmap[person_id], -1)

    return img_copy

In [139]:
def get_limbs_from_keypoints(keypoints):
    limbs = [
        [keypoints.index("right_eye"), keypoints.index("nose")],
        [keypoints.index("right_eye"), keypoints.index("right_ear")],
        [keypoints.index("left_eye"), keypoints.index("nose")],
        [keypoints.index("left_eye"), keypoints.index("left_ear")],
        [keypoints.index("right_shoulder"), keypoints.index("right_elbow")],
        [keypoints.index("right_elbow"), keypoints.index("right_wrist")],
        [keypoints.index("left_shoulder"), keypoints.index("left_elbow")],
        [keypoints.index("left_elbow"), keypoints.index("left_wrist")],
        [keypoints.index("right_hip"), keypoints.index("right_knee")],
        [keypoints.index("right_knee"), keypoints.index("right_ankle")],
        [keypoints.index("left_hip"), keypoints.index("left_knee")],
        [keypoints.index("left_knee"), keypoints.index("left_ankle")],
        [keypoints.index("right_shoulder"), keypoints.index("left_shoulder")],
        [keypoints.index("right_hip"), keypoints.index("left_hip")],
        [keypoints.index("right_shoulder"), keypoints.index("right_hip")],
        [keypoints.index("left_shoulder"), keypoints.index("left_hip")],
    ]
    return limbs

In [140]:
keypoints = ['nose','left_eye','right_eye',\
             'left_ear','right_ear','left_shoulder',\
             'right_shoulder','left_elbow','right_elbow',\
             'left_wrist','right_wrist','left_hip',\
             'right_hip','left_knee', 'right_knee', \
             'left_ankle','right_ankle']

In [141]:
limbs = get_limbs_from_keypoints(keypoints)

In [142]:
def draw_skeleton_per_person(img, all_keypoints, all_scores, confs, thickness, keypoint_threshold=2, conf_threshold=0.9):

    # создаём спектр цветов
    cmap = list(itertools.permutations((200, 100, 0), 3))
    # создаём копию изображений
    img_copy = img.copy()
    # если keypoints детектированы
    if len(all_keypoints)>0:
        # для каждого задетектированного человека
        for person_id in range(len(all_keypoints)):
            # проверяем степень уверенности детектора
            if confs[person_id]>conf_threshold:
            # собираем ключевые точки конкретного человека
                keypoints = all_keypoints[person_id, ...]

                # для каждой конечности
                for limb_id in range(len(limbs)):
                    # отмечаем начало конечности
                    limb_loc1 = keypoints[limbs[limb_id][0], :2].detach().numpy().astype(np.int32)
                    # отмечаем окончание конечности
                    limb_loc2 = keypoints[limbs[limb_id][1], :2].detach().numpy().astype(np.int32)
                    # определяем скор по конечности как минимальный скор среди ключевых точек конечности
                    limb_score = min(all_scores[person_id, limbs[limb_id][0]], all_scores[person_id, limbs[limb_id][1]])
                    # проверяем степень уверенности детектора опорной точки
                    if limb_score> keypoint_threshold:
                        # рисуем линии вдоль конечности
                        cv2.line(img_copy, tuple(limb_loc1), tuple(limb_loc2), cmap[person_id], thickness)

    return img_copy

In [143]:
step_imshow = 3
step_save = 6
nrows = len(list(range(0, num_foto, step_imshow)))

figure, ax = plt.subplots(nrows=nrows, ncols=2, figsize=(15, 6*nrows))

# визуализация каждой третьей секунды обоих видео
for num_rows, foto in zip(range(nrows),range(0, num_foto, step_imshow)):
    path_model = '/content/model/frame'+ str(foto)+'.jpg'
    path_person = '/content/person/frame'+ str(foto)+'.jpg'

    img_arr_model = cv2.imread(path_model)
    img_arr_model = cv2.cvtColor(img_arr_model, cv2.COLOR_BGR2RGB)

    img_arr_person = cv2.imread(path_person)
    img_arr_person = cv2.cvtColor(img_arr_person, cv2.COLOR_BGR2RGB)

    # коуч с опорными точками
    model_with_point = draw_keypoints_per_person(
        img_arr_model,
        all_keypoints = prediction_model[foto]['keypoints'],
        all_scores = prediction_model[foto]['keypoints_scores'],
        confs = prediction_model[foto]['scores'],
        thickness = 5
        )
    
    # коуч с опорными точками и каркасом
    model_with_skeleton = draw_skeleton_per_person(
        model_with_point,
        all_keypoints = prediction_model[foto]['keypoints'],
        all_scores = prediction_model[foto]['keypoints_scores'],
        confs = prediction_model[foto]['scores'],
        thickness = 3
        )
    
    # студент с опорными точками
    person_with_point = draw_keypoints_per_person(
        img_arr_person,
        all_keypoints = prediction_person[foto]['keypoints'],
        all_scores = prediction_person[foto]['keypoints_scores'],
        confs = prediction_person[foto]['scores'],
        thickness = 4
        )
    
    # студент с опорными точками и каркасом
    person_with_skeleton = draw_skeleton_per_person(
        person_with_point,
        all_keypoints = prediction_person[foto]['keypoints'],
        all_scores = prediction_person[foto]['keypoints_scores'],
        confs = prediction_person[foto]['scores'],
        thickness = 2
        )
    
    # вывод данных
    ax[num_rows, 0].imshow(model_with_skeleton)
    ax[num_rows, 1].imshow(person_with_skeleton)
    
    ax[num_rows, 0].set_title("Сoach")
    ax[num_rows, 1].set_title(''.join(['Person. Cosine similarity ',
                                   cos_sim_print[foto],
                                   ' OKS ',
                                   oks_print[foto]]))
    ax[num_rows, 0].set_axis_off()
    ax[num_rows, 1].set_axis_off()

    coach_bgr = cv2.cvtColor(model_with_skeleton, cv2.COLOR_RGB2BGR)
    person_bgr = cv2.cvtColor(person_with_skeleton, cv2.COLOR_RGB2BGR)
    
    # нанесение метрик на отдельные кадры студента
    # и их сохранение вместе с кадром из видео коуча
    if foto % step_save == 0:
      # нанесение метрик на фото студента
      font = cv2.FONT_HERSHEY_SIMPLEX
      cv2.putText(person_bgr,
                  ''.join(['Limb direction ', cos_sim_print[foto]]),
                  (50, 50), 
                  font, 0.5, 
                  (255, 0, 0), 
                  1, 
                  cv2.LINE_4
                  )
      
      cv2.putText(person_bgr,
                  ''.join([' Joint position ', oks_print[foto]]),
                  (50, 75), 
                  font, 0.5, 
                  (255, 0, 0), 
                  1, 
                  cv2.LINE_4
                  )
      # изменение размера изображения коуча
      new_size = person_bgr.shape[:-1][::-1]
      resized_coach = cv2.resize(coach_bgr, new_size,
                                 interpolation = cv2.INTER_AREA)
     # сохранение файла
      folder_name = '/content/tandem/' 
      if not os.path.exists(folder_name):
        os.makedirs(folder_name)
      filename ="coach_person_%d.jpg" % foto
      directory = os.path.join(folder_name, filename)
      tandem_img = np.concatenate((resized_coach, person_bgr),
                                  axis=1)
      cv2.imwrite(directory, tandem_img)
  
    
plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Нанесение метрик на видео

In [144]:
# создание видео из всех изображений студента
cap = cv2.VideoCapture(video_person_path)  
  
# Определение кодека и создание видеозаписи

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)

fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter('output_video.mp4', fourcc, 2.0, size)
id_foto = 0

while id_foto != num_foto:
    ret, frame = cap.read() 
    if(ret):
      path = '/content/person/frame'+ str(id_foto)+'.jpg'
      img_arr = cv2.imread(path)
      prediction = prediction_person[id_foto]

      # студент с опорными точками
      person_with_point = draw_keypoints_per_person(
          img_arr,
          all_keypoints = prediction['keypoints'],
          all_scores = prediction['keypoints_scores'],
          confs = prediction['scores'],
          thickness = 4
          )
      
      # студент с опорными точками и каркасом
      person_with_skeleton = draw_skeleton_per_person(
          person_with_point,
          all_keypoints = prediction['keypoints'],
          all_scores = prediction['keypoints_scores'],
          confs = prediction['scores'],
          thickness = 2
          )
      
      # нанесение метрик на фото
      font = cv2.FONT_HERSHEY_SIMPLEX
      cv2.putText(person_with_skeleton,
                  ''.join(['Limb direction ', cos_sim_print[id_foto]]),
                  (50, 50), 
                  font, 0.5, 
                  (255, 0, 0), 
                  1, 
                  cv2.LINE_4
                  )
      
      cv2.putText(person_with_skeleton,
                  ''.join([' Joint position ', oks_print[id_foto]]),
                  (50, 75), 
                  font, 0.5, 
                  (255, 0, 0), 
                  1, 
                  cv2.LINE_4
                  )
      # запись фрема
      out.write(person_with_skeleton)      
      id_foto+=1
      #cv2_imshow(person_with_skeleton)
        
      # ключ для выхода из записи
      if cv2.waitKey(1) & 0xFF == ord('a'):
          break
  
cap.release()
out.release() 
cv2.destroyAllWindows()

In [145]:
# количество ключевых точек на всех фотографиях
num_kpoints_person = 17*len(id_pose)
# количество значимых ключевых точек
num_cut_kpoints = 0
for foto in range(len(id_pose)):
  num_cut_kpoints += person_keypoints[foto].shape[0]

In [146]:
print(f'Average limb direction = {(stats.mode(cos_sim)[0].item()):.2%}')
print(f'Average joint position = {(stats.mode(oks)[0].item()):.2%}')
print(f'{(len(id_cut)/num_foto):.0%} images were unvalued')
print(f'{(1 - num_cut_kpoints/num_kpoints_person):.0%} key points on valued images were deleted')

Average limb direction = 98.86%
Average joint position = 100.00%
7% images were unvalued
15% key points on valued images were deleted


In [27]:
!zip -r tandem.zip tandem/ 

  adding: tandem/ (stored 0%)
  adding: tandem/coach_person_36.jpg (deflated 0%)
  adding: tandem/coach_person_30.jpg (deflated 0%)
  adding: tandem/coach_person_108.jpg (deflated 0%)
  adding: tandem/coach_person_84.jpg (deflated 0%)
  adding: tandem/coach_person_66.jpg (deflated 0%)
  adding: tandem/coach_person_24.jpg (deflated 0%)
  adding: tandem/coach_person_6.jpg (deflated 0%)
  adding: tandem/coach_person_138.jpg (deflated 0%)
  adding: tandem/coach_person_0.jpg (deflated 0%)
  adding: tandem/coach_person_114.jpg (deflated 0%)
  adding: tandem/coach_person_96.jpg (deflated 0%)
  adding: tandem/coach_person_132.jpg (deflated 0%)
  adding: tandem/coach_person_78.jpg (deflated 0%)
  adding: tandem/coach_person_54.jpg (deflated 0%)
  adding: tandem/coach_person_12.jpg (deflated 0%)
  adding: tandem/coach_person_60.jpg (deflated 0%)
  adding: tandem/coach_person_102.jpg (deflated 0%)
  adding: tandem/coach_person_90.jpg (deflated 0%)
  adding: tandem/coach_person_42.jpg (deflated 0%

In [28]:
!zip -r mp4tojpg_model.zip model/

  adding: model/ (stored 0%)
  adding: model/frame139.jpg (deflated 7%)
  adding: model/frame67.jpg (deflated 6%)
  adding: model/frame129.jpg (deflated 6%)
  adding: model/frame97.jpg (deflated 5%)
  adding: model/frame36.jpg (deflated 6%)
  adding: model/frame69.jpg (deflated 6%)
  adding: model/frame76.jpg (deflated 6%)
  adding: model/frame100.jpg (deflated 6%)
  adding: model/frame12.jpg (deflated 5%)
  adding: model/frame5.jpg (deflated 5%)
  adding: model/frame104.jpg (deflated 5%)
  adding: model/frame145.jpg (deflated 5%)
  adding: model/frame122.jpg (deflated 5%)
  adding: model/frame120.jpg (deflated 7%)
  adding: model/frame82.jpg (deflated 4%)
  adding: model/frame45.jpg (deflated 5%)
  adding: model/frame1.jpg (deflated 6%)
  adding: model/frame6.jpg (deflated 5%)
  adding: model/frame54.jpg (deflated 5%)
  adding: model/frame83.jpg (deflated 6%)
  adding: model/frame42.jpg (deflated 5%)
  adding: model/frame51.jpg (deflated 7%)
  adding: model/frame134.jpg (deflated 7%)


In [29]:
!zip -r mp4tojpg_person.zip person/

  adding: person/ (stored 0%)
  adding: person/frame139.jpg (deflated 0%)
  adding: person/frame67.jpg (deflated 0%)
  adding: person/frame129.jpg (deflated 0%)
  adding: person/frame97.jpg (deflated 0%)
  adding: person/frame36.jpg (deflated 0%)
  adding: person/frame69.jpg (deflated 0%)
  adding: person/frame76.jpg (deflated 0%)
  adding: person/frame100.jpg (deflated 0%)
  adding: person/frame12.jpg (deflated 0%)
  adding: person/frame5.jpg (deflated 0%)
  adding: person/frame104.jpg (deflated 0%)
  adding: person/frame145.jpg (deflated 0%)
  adding: person/frame122.jpg (deflated 0%)
  adding: person/frame120.jpg (deflated 0%)
  adding: person/frame82.jpg (deflated 0%)
  adding: person/frame45.jpg (deflated 0%)
  adding: person/frame1.jpg (deflated 0%)
  adding: person/frame6.jpg (deflated 0%)
  adding: person/frame54.jpg (deflated 0%)
  adding: person/frame83.jpg (deflated 0%)
  adding: person/frame42.jpg (deflated 0%)
  adding: person/frame51.jpg (deflated 0%)
  adding: person/fra